# Predicting Graduation Rates

In [1]:
import pandas as pd
import numpy as np
import sys
pd.options.display.max_columns = 200
pd.options.display.max_rows = 170

## Viewing the dataframe

In [2]:
df = pd.read_csv("data/UniversityData.csv", index_col="UnitID")

In [3]:
df.drop("Unnamed: 136", axis=1, inplace=True)

In [10]:
schema = pd.read_csv("data/Schema.csv")

In [5]:
df.head()

,Institution Name,City location of institution (HD2019),Grand total (S2019_IS All instructional staff),Grand total men (S2019_IS All instructional staff),Grand total women (S2019_IS All instructional staff),American Indian or Alaska Native total (S2019_IS All instructional staff),American Indian or Alaska Native men (S2019_IS All instructional staff),American Indian or Alaska Native women (S2019_IS All instructional staff),Asian total (S2019_IS All instructional staff),Asian men (S2019_IS All instructional staff),Asian women (S2019_IS All instructional staff),Black or African American total (S2019_IS All instructional staff),Black or African American men (S2019_IS All instructional staff),Black or African American women (S2019_IS All instructional staff),Hispanic or Latino total (S2019_IS All instructional staff),Hispanic or Latino men (S2019_IS All instructional staff),Hispanic or Latino women (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander total (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander men (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander women (S2019_IS All instructional staff),White total (S2019_IS All instructional staff),White men (S2019_IS All instructional staff),White women (S2019_IS All instructional staff),Two or more races total (S2019_IS All instructional staff),Two or more races men (S2019_IS All instructional staff),Two or more races women (S2019_IS All instructional staff),Race/ethnicity unknown total (S2019_IS All instructional staff),Race/ethnicity unknown men (S2019_IS All instructional staff),Race/ethnicity unknown women (S2019_IS All instructional staff),Nonresident alien total (S2019_IS All instructional staff),Nonresident alien men (S2019_IS All instructional staff),Nonresident alien women (S2019_IS All instructional staff),Graduation rate total cohort (DRVGR2019),Graduation rate men (DRVGR2019),Graduation rate women (DRVGR2019),Graduation rate American Indian or Alaska Native (DRVGR2019),Graduation rate Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019),Graduation rate Asian (DRVGR2019),Graduation rate Native Hawaiian or Other Pacific Islander (DRVGR2019),Graduation rate Black non-Hispanic (DRVGR2019),Graduation rate Hispanic (DRVGR2019),Graduation rate White non-Hispanic (DRVGR2019),Graduation rate two or more races (DRVGR2019),Graduation rate Race/ethnicity unknown (DRVGR2019),Graduation rate Nonresident alien (DRVGR2019),Transfer-out rate total cohort (DRVGR2019),Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time undergraduates awarded federal state local or institutional grant aid (SFA1819),Core revenues total dollars (GASB) (DRVF2019),Tuition and fees as a percent of core revenues (GASB) (DRVF2019),State appropriations as percent of core revenues (GASB) (DRVF2019),Local appropriations as a percent of core revenues (GASB) (DRVF2019),Government grants and contracts as a percent of core revenues (GASB) (DRVF2019),Private gifts grants and contracts as a percent of core revenues (GASB) (DRVF2019),Investment return as a percent of core revenues (GASB) (DRVF2019),Other revenues as a percent of core revenues (GASB) (DRVF2019),Core revenues total dollars (FASB) (DRVF2019),Tuition and fees as a percent of core revenues (FASB) (DRVF2019),Government grants and contracts as a percent of core revenues (FASB) (DRVF2019),Private gifts grants contracts/contributions from affiliated entities as a percent of core revenues (FASB) (DRVF2019),Investment return as a percent of core revenues (FASB) (DRVF2019),Other revenues as a percent of core revenues (FASB) (DRVF2019),Core revenues total dollars (for-profit institutions) (DRVF2019),Tuition and fees as a percent of core revenues (for-profit institutions) (DRVF2019),Government grants and contracts as a percent of core revenues (for-profit institutions) (DRVF2019),Private gifts grants contracts as a percent of co

In [6]:
df.columns[32]

'Graduation rate  total cohort (DRVGR2019)'

In [7]:
#drop where total graduation rate is missing
df.dropna(subset=[df.columns[32]], inplace=True)

In [8]:
df.shape

(2315, 135)

In [15]:
schema[schema["VariableName"].str.contains("Parent")]

,VariableName,Value,ValueLabel
135,Parent/child indicator - Finance (FLAGS2019),1,Parent record - includes data from branch camp...
136,Parent/child indicator - Finance (FLAGS2019),2,Child record - data reported with parent campus
137,Parent/child indicator - Finance (FLAGS2019),3,Partial child record - reports revenues/expens...
138,Parent/child indicator - Finance (FLAGS2019),5,Child record - reports partial data but other ...
139,Parent/child indicator - Finance (FLAGS2019),-2,Not applicable


## Spliting data frame into dependent variables, categorical and ordinal features, and numerical features

In [9]:
#list of categorical features
cat_ordinal_groups = list(schema.groupby("VariableName").groups.keys())

In [10]:
categorical_df = df.loc[:,cat_ordinal_groups] #dataframe with categorical features

In [11]:
df.columns[32:46]

Index(['Graduation rate  total cohort (DRVGR2019)',
       'Graduation rate  men (DRVGR2019)',
       'Graduation rate  women (DRVGR2019)',
       'Graduation rate  American Indian or Alaska Native (DRVGR2019)',
       'Graduation rate  Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019)',
       'Graduation rate  Asian (DRVGR2019)',
       'Graduation rate  Native Hawaiian or Other Pacific Islander (DRVGR2019)',
       'Graduation rate  Black  non-Hispanic (DRVGR2019)',
       'Graduation rate  Hispanic (DRVGR2019)',
       'Graduation rate  White  non-Hispanic (DRVGR2019)',
       'Graduation rate  two or more races (DRVGR2019)',
       'Graduation rate  Race/ethnicity unknown (DRVGR2019)',
       'Graduation rate  Nonresident alien (DRVGR2019)',
       'Transfer-out rate  total cohort (DRVGR2019)'],
      dtype='object')

In [12]:
#dependent features list
dependent_var_list = list(df.columns[32:46])

In [13]:
dependent_df = df.loc[:,dependent_var_list]

In [14]:
#Numeric Data Frame will be all of those columns outside of dependent and categorical/ordinal

In [15]:
not_numeric = cat_ordinal_groups + dependent_var_list

In [16]:
numeric_df = df.drop(not_numeric, axis=1)

# Viewing dataframe by dtypes

In [17]:
numeric_df.describe()

,Grand total (S2019_IS All instructional staff),Grand total men (S2019_IS All instructional staff),Grand total women (S2019_IS All instructional staff),American Indian or Alaska Native total (S2019_IS All instructional staff),American Indian or Alaska Native men (S2019_IS All instructional staff),American Indian or Alaska Native women (S2019_IS All instructional staff),Asian total (S2019_IS All instructional staff),Asian men (S2019_IS All instructional staff),Asian women (S2019_IS All instructional staff),Black or African American total (S2019_IS All instructional staff),Black or African American men (S2019_IS All instructional staff),Black or African American women (S2019_IS All instructional staff),Hispanic or Latino total (S2019_IS All instructional staff),Hispanic or Latino men (S2019_IS All instructional staff),Hispanic or Latino women (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander total (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander men (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander women (S2019_IS All instructional staff),White total (S2019_IS All instructional staff),White men (S2019_IS All instructional staff),White women (S2019_IS All instructional staff),Two or more races total (S2019_IS All instructional staff),Two or more races men (S2019_IS All instructional staff),Two or more races women (S2019_IS All instructional staff),Race/ethnicity unknown total (S2019_IS All instructional staff),Race/ethnicity unknown men (S2019_IS All instructional staff),Race/ethnicity unknown women (S2019_IS All instructional staff),Nonresident alien total (S2019_IS All instructional staff),Nonresident alien men (S2019_IS All instructional staff),Nonresident alien women (S2019_IS All instructional staff),Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time undergraduates awarded federal state local or institutional grant aid (SFA1819),Core revenues total dollars (GASB) (DRVF2019),Tuition and fees as a percent of core revenues (GASB) (DRVF2019),State appropriations as percent of core revenues (GASB) (DRVF2019),Local appropriations as a percent of core revenues (GASB) (DRVF2019),Government grants and contracts as a percent of core revenues (GASB) (DRVF2019),Private gifts grants and contracts as a percent of core revenues (GASB) (DRVF2019),Investment return as a percent of core revenues (GASB) (DRVF2019),Other revenues as a percent of core revenues (GASB) (DRVF2019),Core revenues total dollars (FASB) (DRVF2019),Tuition and fees as a percent of core revenues (FASB) (DRVF2019),Government grants and contracts as a percent of core revenues (FASB) (DRVF2019),Private gifts grants contracts/contributions from affiliated entities as a percent of core revenues (FASB) (DRVF2019),Investment return as a percent of core revenues (FASB) (DRVF2019),Other revenues as a percent of core revenues (FASB) (DRVF2019),Core revenues total dollars (for-profit institutions) (DRVF2019),Tuition and fees as a percent of core revenues (for-profit institutions) (DRVF2019),Government grants and contracts as a percent of core revenues (for-profit institutions) (DRVF2019),Private gifts grants contracts as a percent of core revenues (for-profit institutions) (DRVF2019),Investment return as a percent of core revenues (for-profit institutions) (DRVF2019),Sales and services of educational activities as a percent of core revenues (for-profit institutions) (DRVF2019),Other revenues as a percent of core revenues (for-profit institutions) (DRVF2019),Core expenses total dollars (GASB) (DRVF2019),Instruction expenses as a percent of total core expenses (GASB) (DRVF2019),Research expenses as a percent of total core expenses (GASB) (DRVF2019),Public service expenses as a percent of total core expenses (GASB) (DRVF2019),Academic support expenses as a percent of total core expenses (GASB) (DRVF2019),Student service expenses as a percent 

In [18]:
dependent_df.reset_index(inplace=True)

In [19]:
graduation_df = dependent_df.iloc[:,:2]

In [20]:
graduation_df

,UnitID,Graduation rate total cohort (DRVGR2019)
0,180203,29.0
1,222178,61.0
2,138558,26.0
3,172866,0.0
4,108232,45.0
...,...,...
2310,217059,63.0
2311,141361,42.0
2312,206695,41.0
2313,204255,54.0


In [21]:
categorical_df

,Carnegie Classification 2018: Basic (HD2018),Carnegie Classification 2018: Enrollment Profile (HD2018),Carnegie Classification 2018: Size and Setting (HD2018),Historically Black College or University (HD2018),Institution size category (HD2018),Parent/child indicator - Finance (FLAGS2019),Sector of institution (HD2018),State abbreviation (HD2018)
UnitID,,,,,,,,
180203,33.0,1.0,1.0,2.0,1.0,-2.0,4.0,MT
222178,18.0,4.0,14.0,2.0,3.0,-2.0,2.0,TX
138558,23.0,2.0,10.0,2.0,2.0,-2.0,1.0,GA
172866,28.0,2.0,6.0,2.0,1.0,-2.0,3.0,MN
108232,18.0,5.0,12.0,2.0,4.0,-2.0,3.0,CA
...,...,...,...,...,...,...,...,...
217059,20.0,3.0,14.0,2.0,2.0,-2.0,2.0,PA
141361,21.0,2.0,11.0,2.0,2.0,-2.0,2.0,GA
206695,18.0,3.0,15.0,2.0,4.0,-2.0,1.0,OH


# Data Wrangling - Filling Missing Values

### Cleaning for baseline model: Categorical

In [22]:
categorical_df.fillna("isNaN", inplace=True)

In [23]:
categorical_df = categorical_df.astype("string")

In [24]:
for i in categorical_df.columns:
    categorical_df[i] = categorical_df[i].str.replace(".0", "")

In [25]:
categorical_df.iloc[:,1].unique()

<StringArray>
['1', '4', '2', '5', '6', '3', 'isNaN', '-2', '7']
Length: 9, dtype: string

In [26]:
maps = {}

for i in cat_ordinal_groups:    
    dict_to_add = {}
   
    val_label_df = schema[schema["VariableName"] == i]

    valLabelDf = val_label_df[["Value", "ValueLabel"]]


    val_label_dict = valLabelDf.set_index("Value").transpose().to_dict()

    for j,v in val_label_dict.items():

        val_label_dict[j] = v['ValueLabel']
    
    maps[i] = val_label_dict
    
for i in cat_ordinal_groups:
    categorical_df[i].replace(maps[i], inplace=True)
    

    

In [27]:
categorical_df["City location of institution (HD2019)"] = df[df.columns[1]]

In [28]:
#for time and simplicity sake for our baseline model we will drop city because it has over 1000 unique values
categorical_df.head()

,Carnegie Classification 2018: Basic (HD2018),Carnegie Classification 2018: Enrollment Profile (HD2018),Carnegie Classification 2018: Size and Setting (HD2018),Historically Black College or University (HD2018),Institution size category (HD2018),Parent/child indicator - Finance (FLAGS2019),Sector of institution (HD2018),State abbreviation (HD2018),City location of institution (HD2019)
UnitID,,,,,,,,,
180203,Tribal Colleges,Exclusively undergraduate two-year,"Two-year, very small",No,"Under 1,000",Not applicable,"Public, 2-year",Montana,Harlem
222178,Master's Colleges & Universities: Larger Programs,High undergraduate,"Four-year, medium, highly residential",No,"5,000 - 9,999",Not applicable,"Private not-for-profit, 4-year or above",Texas,Abilene
138558,Baccalaureate/Associate's Colleges: Mixed Bacc...,Exclusively undergraduate four-year,,No,"1,000 - 4,999",Not applicable,"Public, 4-year or above",Georgia,Tifton
172866,Special Focus Four-Year: Other Technology-Rela...,Exclusively undergraduate four-year,"Four-year, very small, primarily nonresidential",No,"Under 1,000",Not applicable,"Private for-profit, 4-year or above",Minnesota,Bloomington
108232,Master's Colleges & Universities: Larger Programs,Majority undergraduate,"Four-year, medium, primarily nonresidential",No,"10,000 - 19,999",Not applicable,"Private for-profit, 4-year or above",California,San Francisco


In [29]:
categorical_df_no_city = categorical_df.drop(
    categorical_df.columns[-1],
    axis=1
)

# Cleaning for baseline model: Numeric 
## Merging Financial Data

In [30]:
from functools import reduce

In [31]:
def merge_into_mini(dframe, list_of_cols, final_col_name):
    df_list = []
    for i in list_of_cols:
        df_1 = pd.DataFrame(dframe.copy(deep=True)[i])
        df_1.reset_index(inplace=True)
        df_1.dropna(inplace=True)
        df_1.columns = ["UnitID", final_col_name]
        df_list.append(df_1)
    df_final = pd.concat(df_list)
    return df_final

In [32]:
numeric_df.columns[55:75]

Index(['Core expenses  total dollars (GASB) (DRVF2019)',
       'Instruction expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Research expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Public service expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Academic support expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Student service expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Institutional support expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Other core expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Core expenses  total dollars (FASB) (DRVF2019)',
       'Instruction expenses as a percent of total core expenses (FASB) (DRVF2019)',
       'Research expenses as a percent of total core expenses (FASB) (DRVF2019)',
       'Public service expenses as a percent of total core expenses (FASB) (DRVF2019)',
       'Academic s

In [33]:
#create expenses classes
class Container:
    def __init__(self, col_name):
        self.main_list = []
        self.name = col_name.title().replace(" ", "_")
        for i in numeric_df.columns:
            if col_name in i:
                self.main_list.append(i)
    def __repr__(self):
        return self.name

expenses_list = [
    Container("Core expenses"), Container("Instruction expenses"), 
    Container("Research expenses"), Container("Public service expenses"),
    Container("Academic support expenses"),
    Container("Student service expenses"), Container("Institutional support expenses"),
    Container("Other core expenses")
]

revenues_list = [
    Container("Core revenues"), Container("Tuition and fees"),
    Container("Government grants"), Container("Private gifts"),
    Container("Investment return"), Container("Sales and services"),
    Container("Other revenues")
    
]
                   

In [34]:
expenses_df_list = [merge_into_mini(numeric_df, i.main_list, repr(i)) for i in expenses_list]

In [35]:
expenses_df = reduce(lambda x, y: pd.merge(x, y, on = 'UnitID', how="outer"), expenses_df_list)

In [36]:
revenues_df_list = [merge_into_mini(numeric_df, i.main_list, repr(i)) for i in revenues_list]    

In [37]:
revenues_df = reduce(lambda x, y: pd.merge(x, y, on = 'UnitID', how="outer"), revenues_df_list)

In [38]:
revenues_df.fillna(value=0, inplace=True)

In [39]:
#converting to dollar amounts instead of percentages
'''
for i in revenues_df.columns[2:]:
    revenues_df[i] = revenues_df[i].apply(lambda x: x / 100) * revenues_df["Core_Revenues"]

'''


'\nfor i in revenues_df.columns[2:]:\n    revenues_df[i] = revenues_df[i].apply(lambda x: x / 100) * revenues_df["Core_Revenues"]\n\n'

In [40]:
revenues_df.head()

,UnitID,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues
0,180203,7078959.0,2.0,60.0,7.0,1.0,0.0,29.0
1,138558,70491879.0,17.0,17.0,2.0,0.0,0.0,28.0
2,126182,52099218.0,34.0,51.0,5.0,1.0,0.0,9.0
3,100654,143112797.0,33.0,30.0,2.0,0.0,0.0,6.0
4,100724,134993142.0,30.0,25.0,2.0,1.0,0.0,7.0


In [41]:
expenses_df.fillna(value=0, inplace=True)

In [42]:
#converting to dollar amounts instead of percentages
'''
for i in expenses_df.columns[2:]:
    expenses_df[i] = expenses_df[i].apply(lambda x: x / 100) * expenses_df["Core_Expenses"]
'''


'\nfor i in expenses_df.columns[2:]:\n    expenses_df[i] = expenses_df[i].apply(lambda x: x / 100) * expenses_df["Core_Expenses"]\n'

In [43]:
expenses_df.head()

,UnitID,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses
0,180203,7294344.0,29.0,3.0,7.0,3.0,8.0,16.0,36.0
1,138558,50201124.0,42.0,0.0,3.0,15.0,9.0,19.0,13.0
2,126182,35725137.0,53.0,0.0,3.0,7.0,18.0,15.0,5.0
3,100654,146878227.0,22.0,9.0,9.0,6.0,14.0,11.0,29.0
4,100724,134888910.0,29.0,3.0,4.0,9.0,12.0,27.0,16.0


In [44]:
finances_df = revenues_df.merge(expenses_df, how="outer", on="UnitID")

In [45]:
finances_df.head()

,UnitID,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses
0,180203,7078959.0,2.0,60.0,7.0,1.0,0.0,29.0,7294344.0,29.0,3.0,7.0,3.0,8.0,16.0,36.0
1,138558,70491879.0,17.0,17.0,2.0,0.0,0.0,28.0,50201124.0,42.0,0.0,3.0,15.0,9.0,19.0,13.0
2,126182,52099218.0,34.0,51.0,5.0,1.0,0.0,9.0,35725137.0,53.0,0.0,3.0,7.0,18.0,15.0,5.0
3,100654,143112797.0,33.0,30.0,2.0,0.0,0.0,6.0,146878227.0,22.0,9.0,9.0,6.0,14.0,11.0,29.0
4,100724,134993142.0,30.0,25.0,2.0,1.0,0.0,7.0,134888910.0,29.0,3.0,4.0,9.0,12.0,27.0,16.0


## Staff Diversity

<ol>
    <li>All expressed as percentages</li>
    <li>Split between Black, White, Hispanic/Latino, Asian, Other</li>
</ol>

In [46]:
staff_df = numeric_df.loc[:,numeric_df.columns[2:32]]

In [47]:
staff_df.dropna(subset=[staff_df.columns[0]], inplace=True)

In [48]:
staff_df.reset_index(inplace=True)

In [49]:
for i in staff_df.columns[2:]:
    staff_df[i] = (staff_df[i] / staff_df[staff_df.columns[1]]) * 100

In [50]:
staff_df = staff_df.round(2)

In [51]:
staff_df.head()

,UnitID,Grand total (S2019_IS All instructional staff),Grand total men (S2019_IS All instructional staff),Grand total women (S2019_IS All instructional staff),American Indian or Alaska Native total (S2019_IS All instructional staff),American Indian or Alaska Native men (S2019_IS All instructional staff),American Indian or Alaska Native women (S2019_IS All instructional staff),Asian total (S2019_IS All instructional staff),Asian men (S2019_IS All instructional staff),Asian women (S2019_IS All instructional staff),Black or African American total (S2019_IS All instructional staff),Black or African American men (S2019_IS All instructional staff),Black or African American women (S2019_IS All instructional staff),Hispanic or Latino total (S2019_IS All instructional staff),Hispanic or Latino men (S2019_IS All instructional staff),Hispanic or Latino women (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander total (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander men (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander women (S2019_IS All instructional staff),White total (S2019_IS All instructional staff),White men (S2019_IS All instructional staff),White women (S2019_IS All instructional staff),Two or more races total (S2019_IS All instructional staff),Two or more races men (S2019_IS All instructional staff),Two or more races women (S2019_IS All instructional staff),Race/ethnicity unknown total (S2019_IS All instructional staff),Race/ethnicity unknown men (S2019_IS All instructional staff),Race/ethnicity unknown women (S2019_IS All instructional staff),Nonresident alien total (S2019_IS All instructional staff),Nonresident alien men (S2019_IS All instructional staff),Nonresident alien women (S2019_IS All instructional staff)
0,180203,13.0,53.85,46.15,61.54,30.77,30.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38.46,23.08,15.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,222178,265.0,58.11,41.89,0.38,0.38,0.00,1.13,0.75,0.38,5.28,2.64,2.64,4.91,3.02,1.89,0.00,0.00,0.00,86.04,49.43,36.60,1.13,0.75,0.38,0.00,0.00,0.00,1.13,1.13,0.00
2,138558,149.0,51.68,48.32,0.00,0.00,0.00,6.71,3.36,3.36,6.71,4.70,2.01,2.01,1.34,0.67,0.00,0.00,0.00,83.22,41.61,41.61,0.00,0.00,0.00,0.67,0.67,0.00,0.67,0.00,0.67
3,108232,219.0,56.62,43.38,0.00,0.00,0.00,10.05,2.74,7.31,1.83,1.37,0.46,3.65,2.28,1.37,1.37,0.46,0.91,64.38,38.36,26.03,0.91,0.46,0.46,17.81,10.96,6.85,0.00,0.00,0.00
4,126182,103.0,44.66,55.34,0.00,0.00,0.00,0.97,0.97,0.00,0.97,0.00,0.97,16.50,5.83,10.68,1.94,0.97,0.97,76.70,34.95,41.75,0.00,0.00,0.00,2.91,1.94,0.97,0.00,0.00,0.00


For our Study will use totals for simplicity. 

In [52]:
totals_list = ["UnitID"]
for i in staff_df.columns:
    if "total" in i:
        totals_list.append(i)
        
totals_list

['UnitID',
 'Grand total (S2019_IS  All instructional staff)',
 'Grand total men (S2019_IS  All instructional staff)',
 'Grand total women (S2019_IS  All instructional staff)',
 'American Indian or Alaska Native total (S2019_IS  All instructional staff)',
 'Asian total (S2019_IS  All instructional staff)',
 'Black or African American total (S2019_IS  All instructional staff)',
 'Hispanic or Latino total (S2019_IS  All instructional staff)',
 'Native Hawaiian or Other Pacific Islander total (S2019_IS  All instructional staff)',
 'White total (S2019_IS  All instructional staff)',
 'Two or more races total (S2019_IS  All instructional staff)',
 'Race/ethnicity unknown total (S2019_IS  All instructional staff)',
 'Nonresident alien total (S2019_IS  All instructional staff)']

In [53]:
staff_df = staff_df.loc[:,totals_list]

In [54]:
staff_df.head()

,UnitID,Grand total (S2019_IS All instructional staff),Grand total men (S2019_IS All instructional staff),Grand total women (S2019_IS All instructional staff),American Indian or Alaska Native total (S2019_IS All instructional staff),Asian total (S2019_IS All instructional staff),Black or African American total (S2019_IS All instructional staff),Hispanic or Latino total (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander total (S2019_IS All instructional staff),White total (S2019_IS All instructional staff),Two or more races total (S2019_IS All instructional staff),Race/ethnicity unknown total (S2019_IS All instructional staff),Nonresident alien total (S2019_IS All instructional staff)
0,180203,13.0,53.85,46.15,61.54,0.00,0.00,0.00,0.00,38.46,0.00,0.00,0.00
1,222178,265.0,58.11,41.89,0.38,1.13,5.28,4.91,0.00,86.04,1.13,0.00,1.13
2,138558,149.0,51.68,48.32,0.00,6.71,6.71,2.01,0.00,83.22,0.00,0.67,0.67
3,108232,219.0,56.62,43.38,0.00,10.05,1.83,3.65,1.37,64.38,0.91,17.81,0.00
4,126182,103.0,44.66,55.34,0.00,0.97,0.97,16.50,1.94,76.70,0.00,2.91,0.00


## Pell Grant Recipients


In [55]:
grant_df = numeric_df.loc[:,numeric_df.columns[32:34]].reset_index()

## Cohort Diversity

In [56]:
cohort_diversity_df = numeric_df.loc[:,numeric_df.columns[79:90]].reset_index()

## Age and Size of cohort

In [57]:
age_df = numeric_df.loc[:,numeric_df.columns[90:96]]

In [58]:
age_df["Incoming_Class_Grand_Total"] = age_df.iloc[:,0] + age_df.iloc[:,3]

In [59]:
for i in age_df.columns[:-1]:
    age_df[i] = age_df[i] / age_df["Incoming_Class_Grand_Total"]

In [60]:
age_df = age_df.round(2)

In [61]:
age_df.isna().sum()

Grand total (EF2013B_RV  Undergraduate  Age under 25 total)       101
Total men (EF2013B_RV  Undergraduate  Age under 25 total)         101
Total women (EF2013B_RV  Undergraduate  Age under 25 total)       101
Grand total (EF2013B_RV  Undergraduate  Age 25 and over total)    101
Total men (EF2013B_RV  Undergraduate  Age 25 and over total)      101
Total women (EF2013B_RV  Undergraduate  Age 25 and over total)    101
Incoming_Class_Grand_Total                                        101
dtype: int64

## Price 

In [62]:
price_df = numeric_df.loc[:,numeric_df.columns[98]].reset_index()

In [63]:
price_df

,UnitID,Total price for out-of-state students living on campus 2012-13 (DRVIC2012_RV)
0,180203,NaN
1,222178,39900.0
2,138558,22684.0
3,172866,NaN
4,108232,36884.0
...,...,...
2310,217059,28820.0
2311,141361,34093.0
2312,206695,20580.0
2313,204255,NaN


## Student Faculty Ratio 

In [64]:
student_faculty = numeric_df.loc[:,numeric_df.columns[105]].reset_index()

In [65]:
student_faculty.head()

,UnitID,Student-to-faculty ratio (EF2019D)
0,180203,10.0
1,222178,15.0
2,138558,17.0
3,172866,9.0
4,108232,14.0


## SAT Scores

In [66]:
test_scores = numeric_df.loc[:,numeric_df.columns[106:]].reset_index()

In [67]:
test_scores.drop(test_scores.columns[5:], axis = 1, inplace=True)

In [68]:
test_scores.drop(test_scores.columns[3], axis=1, inplace=True)

# Merging all cleaned dataframes on UnitId

In [69]:
cleaned_df_list = [
    finances_df, staff_df, grant_df, 
    cohort_diversity_df, age_df, price_df, 
    student_faculty, test_scores,
    categorical_df_no_city, graduation_df
]

In [70]:
pre_dummies_df = reduce(lambda x, y: pd.merge(x, y, on = 'UnitID', how="outer"), cleaned_df_list)

In [71]:
pre_dummies_df.iloc[:,-3].unique()

array(['Public, 2-year', 'Public, 4-year or above',
       'Private not-for-profit, 4-year or above', 'isNaN',
       'Private not-for-profit, 2-year',
       'Private for-profit, 4-year or above',
       'Private for-profit, 2-year',
       'Private for-profit, less-than 2-year'], dtype=object)

In [72]:
pre_imputed = pd.get_dummies(pre_dummies_df, drop_first=True)

In [73]:
pre_imputed.head(2)

,UnitID,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Grand total (S2019_IS All instructional staff),Grand total men (S2019_IS All instructional staff),Grand total women (S2019_IS All instructional staff),American Indian or Alaska Native total (S2019_IS All instructional staff),Asian total (S2019_IS All instructional staff),Black or African American total (S2019_IS All instructional staff),Hispanic or Latino total (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander total (S2019_IS All instructional staff),White total (S2019_IS All instructional staff),Two or more races total (S2019_IS All instructional staff),Race/ethnicity unknown total (S2019_IS All instructional staff),Nonresident alien total (S2019_IS All instructional staff),Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time undergraduates awarded federal state local or institutional grant aid (SFA1819),Percent of total enrollment that are American Indian or Alaska Native (DRVEF2013_RV),Percent of total enrollment that are Asian/Native Hawaiian/Pacific Islander (DRVEF2013_RV),Percent of total enrollment that are Asian (DRVEF2013_RV),Percent of total enrollment that are Native Hawaiian or Other Pacific Islander (DRVEF2013_RV),Percent of total enrollment that are Black or African American (DRVEF2013_RV),Percent of total enrollment that are Hispanic/Latino (DRVEF2013_RV),Percent of total enrollment that are White (DRVEF2013_RV),Percent of total enrollment that are Race/ethnicity unknown (DRVEF2013_RV),Percent of total enrollment that are Nonresident Alien (DRVEF2013_RV),Percent of total enrollment that are two or more races (DRVEF2013_RV),Percent of total enrollment that are women (DRVEF2013_RV),Grand total (EF2013B_RV Undergraduate Age under 25 total),Total men (EF2013B_RV Undergraduate Age under 25 total),Total women (EF2013B_RV Undergraduate Age under 25 total),Grand total (EF2013B_RV Undergraduate Age 25 and over total),Total men (EF2013B_RV Undergraduate Age 25 and over total),Total women (EF2013B_RV Undergraduate Age 25 and over total),Incoming_Class_Grand_Total,Total price for out-of-state students living on campus 2012-13 (DRVIC2012_RV),Student-to-faculty ratio (EF2019D),SAT Critical Reading 75th percentile score (IC2012_RV),SAT Math 75th percentile score (IC2012_RV),ACT Composite 75th percentile score (IC2012_RV),Graduation rate total cohort (DRVGR2019),Carnegie Classification 2018: Basic (HD2018)_Associate's Colleges: High Transfer-High Traditional,Carnegie Classification 2018: Basic (HD2018)_Associate's Colleges: High Transfer-Mixed Traditional/Nontraditional,Carnegie Classification 2018: Basic (HD2018)_Associate's Colleges: High Vocational & Technical-High Nontraditional,Carnegie Classification 2018: Basic (HD2018)_Associate's Colleges: High Vocational & Technical-High Traditional,Carnegie Classification 2018: Basic (HD2018)_Associate's Colleges: High Vocational & Technical-Mixed Traditional/Nontraditional,Carnegie Classification 2018: Basic (HD2018)_Associate's Colleges: Mixed Transfer/Vocational & Technical-High Nontraditional,Carnegie Classification 2018: Basic (HD2018)_Associate's Colleges: Mixed Transfer/Vocational & Technical-High Traditional,Carnegie Classification 2018: Basic (HD2018)_Associate's Colleges: Mixed Transfer/Vocational & Technical-Mixed Traditional/Nontraditional,Carnegie Classification 2018: Basic (HD2018)_Baccalaureate Colleges: Arts & Sciences Focus,Carnegie Classification 2018: Basic (HD2018)_Baccalaureate Colleges: Diverse Fields,Carnegie Classification 2018: Basic (HD2018)_Baccalaureate/Associate's Colleges: Associate's Dominant,Carnegie Classification 2018: Basic (HD2018)_Baccalaureate/Associate's College

In [74]:
graduation_string = pre_imputed.columns[53]

# Finding The Best Imputation Technique

In [75]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.pipeline import make_pipeline, Pipeline

In [76]:
x = pre_imputed.drop(graduation_string, axis=1)
y = pre_imputed[graduation_string]

In [77]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

### Simple Imputation

In [78]:
simple_pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('variance', VarianceThreshold(threshold=0.25)),
    ('selector', SelectKBest(k=75)),
    ('classifier', Lasso(alpha=0.1))
])

In [79]:
simple_pipeline.fit(x_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('variance', VarianceThreshold(threshold=0.25)),
                ('selector', SelectKBest(k=75)),
                ('classifier', Lasso(alpha=0.1))])

In [80]:
# scaler = StandardScaler()
# x_test_scaled = scaler.fit_transform(x_test)

In [81]:
y_pred_simple = simple_pipeline.predict(x_test)

In [82]:
mean_squared_error(y_pred_simple, y_test)

225.0461498566603

### Iterative Imputation

In [83]:
iterative_pipeline = Pipeline([
    ('imputer', IterativeImputer()),
    ('scaler', StandardScaler()),
    ('variance', VarianceThreshold(threshold=0.25)),
    ('selector', SelectKBest(k=75)),
    ('classifier', Lasso(alpha=0.1))
])

In [84]:
iterative_pipeline.fit(x_train, y_train)

Pipeline(steps=[('imputer', IterativeImputer()), ('scaler', StandardScaler()),
                ('variance', VarianceThreshold(threshold=0.25)),
                ('selector', SelectKBest(k=75)),
                ('classifier', Lasso(alpha=0.1))])

In [85]:
y_pred_itr = iterative_pipeline.predict(x_test)

In [86]:
mean_squared_error(y_pred_itr, y_test)

221.74766026911922

### KNN Imputation

In [87]:
KNN_pipeline = Pipeline([
    ('imputer', KNNImputer()),
    ('scaler', StandardScaler()),
    ('variance', VarianceThreshold(threshold=0.25)),
    ('selector', SelectKBest(k=75)),
    ('classifier', Lasso(alpha=0.1))
])

In [88]:
KNN_pipeline.fit(x_train, y_train)

Pipeline(steps=[('imputer', KNNImputer()), ('scaler', StandardScaler()),
                ('variance', VarianceThreshold(threshold=0.25)),
                ('selector', SelectKBest(k=75)),
                ('classifier', Lasso(alpha=0.1))])

In [89]:
y_pred_knn = KNN_pipeline.predict(x_test)

In [90]:
mean_squared_error(y_test, y_pred_knn)

235.37759570087027

In [91]:
# KNN_pipeline.named_steps["classifier"].score(x_test, y_test)
explained_variance_score(y_test, y_pred_knn)

0.49134547423505726

### KNN imputation gives us the best model performence so lets create our final cleaned dataframe with KNN

In [92]:
x.isna().sum()

UnitID                                       0
Core_Revenues                                4
Tuition_And_Fees                             4
Government_Grants                            4
Private_Gifts                                4
                                            ..
State abbreviation (HD2018)_Washington       0
State abbreviation (HD2018)_West Virginia    0
State abbreviation (HD2018)_Wisconsin        0
State abbreviation (HD2018)_Wyoming          0
State abbreviation (HD2018)_isNaN            0
Length: 186, dtype: int64

In [93]:
KNN_imputer = KNNImputer()

In [94]:
x.loc[:,:] = KNN_imputer.fit_transform(x)

In [95]:
x.isna().sum()

UnitID                                       0
Core_Revenues                                0
Tuition_And_Fees                             0
Government_Grants                            0
Private_Gifts                                0
                                            ..
State abbreviation (HD2018)_Washington       0
State abbreviation (HD2018)_West Virginia    0
State abbreviation (HD2018)_Wisconsin        0
State abbreviation (HD2018)_Wyoming          0
State abbreviation (HD2018)_isNaN            0
Length: 186, dtype: int64

In [96]:
x[graduation_string] = y

### We will be studying total graduation rates along with African American Graduation Rates

In [97]:
black_grad_rates_df = pd.DataFrame(dependent_df[["UnitID", dependent_df.columns[8]]])
black_grad_rates_df.head()

,UnitID,Graduation rate Black non-Hispanic (DRVGR2019)
0,180203,NaN
1,222178,53.0
2,138558,12.0
3,172866,0.0
4,108232,20.0


In [98]:
cleaned_df = x.merge(black_grad_rates_df, on="UnitID", how="outer")

In [99]:
cleaned_df.set_index("UnitID", inplace=True)

In [100]:
cleaned_df.to_csv("data/cleaned_df.csv")